below cell has entire solution for trading only position 26 (index 25). In the 4th cell I am trying to set the upper bound dynamically by using an RSI limit value with only 10% of RSI values for that company above that value, which roughly corresponds with the hardcoded value used -> 70. This strategy should work roughly equally accross different companies

In [2]:
# mean(PL): 11.9
# return: 0.02521
# StdDev(PL): 81.29
# annSharpe(PL): 2.32 
# totDvolume: 118532 
# Score: 3.78

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5
# multiplier = 3
# max_pos = 10000
# no shorting, but closing position at upper limit

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5
multiplier = 3
max_pos = 10000

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i])[-1] for i in range(nins)])

    # Create the new position array
    new_pos = np.zeros(nins)

    c26_rsi = rsi_values[25]

    # #buy if RSI < RSI_lower_bound
    # if c26_rsi <= RSI_lower_bound:
    #     new_pos[25] = max_pos

    #maintain if RSI < middle bound
    if c26_rsi < RSI_middle_bound:      #changed from elif with above
        # print("maintain long activated")
        new_pos[25] = (RSI_middle_bound-c26_rsi) * multiplier * max_pos / 100

    #short if RSI above upper limit / without shorting this just closes our position
    elif c26_rsi >= RSI_upper_bound:
        new_pos[25] = 0 #-max_pos - no shorting
    
    #maintain if RSI > middle bound
    elif c26_rsi > RSI_middle_bound:
        # print("maintain short activated")
        new_pos[25] = currentPos[25] #(RSI_upper_bound-c26_rsi) * multiplier * max_pos / 100
    
    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = max_pos * change

    # if position_changes[25] != 0:
    #     c26price = prcSoFar[25][-1]
    #     print(f"RSI: {c26_rsi}, price: {c26price}, existing pos: {currentPos[25]}, new_pos: {new_pos[25]}, max_change: +-{max_change}")

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

import pandas as pd

nInst = 50
nt = 500
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.shape
    return df.values.T

pricesFile="./prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(250,501): # 250 in training period here 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] 
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        #print(t)
        #print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)

Loaded 50 instruments for 500 days
=====
mean(PL): 11.9
return: 0.02521
StdDev(PL): 81.29
annSharpe(PL): 2.32 
totDvolume: 118532 
Score: 3.78


### Calculating relative volatility

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv('prices.txt', header=None, sep="   ")
# df = pd.DataFrame(data=data, columns=columns)
headers = [f'c{i}' for i in range(1, 51)]
df.columns = headers

# Calculate the mean and standard deviation for each column
mean = df.mean()
std_dev = df.std()

# Calculate the coefficient of variation (CV) for each column
cv = std_dev / mean

# Create a DataFrame for description including volatility (CV)
description = df.describe()
description.loc['volatility'] = cv

# Identify the 5 most volatile columns based on CV
most_volatile_columns = cv.nlargest(5)

# Display the 5 most volatile columns
print("\n5 Most Volatile Columns:")
print(most_volatile_columns)

least_volatile_columns = cv.nsmallest(5)

# Display the 5 most volatile columns
print("\n5 Least Volatile Columns:")
print(least_volatile_columns)


5 Most Volatile Columns:
c26    0.182096
c36    0.151494
c42    0.123849
c12    0.092820
c49    0.092431
dtype: float64

5 Least Volatile Columns:
c40    0.004706
c29    0.007532
c9     0.008548
c18    0.010238
c39    0.011873
dtype: float64


C:\Users\harol\AppData\Local\Temp\ipykernel_10628\2709598317.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('prices.txt', header=None, sep="   ")


In [25]:
threshold = 0.01
change = 0.1
max_pos_change = 10000 * change

### Trading Pos 26

In [35]:
# mean(PL): 11.9
# return: 0.02521
# StdDev(PL): 81.29
# annSharpe(PL): 2.32 
# totDvolume: 118532 
# Score: 3.78

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5
# multiplier = 3
# max_pos = 10000
# no shorting, but closing position at upper limit

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
# RSI_upper_bound = 70
# RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5
multiplier = 3
max_pos = 10000
prop_days_over_upper_bound = 0.1
prop_days_under_lower_bound = 0.05

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

# def calculate_dynamic_rsi_bounds(rsi_values, prop_days_over_upper_bound, prop_days_under_lower_bound):
#     sorted_rsi = np.sort(rsi_values)
#     n = len(sorted_rsi)
    
#     upper_index = int((1 - prop_days_over_upper_bound) * n)
#     lower_index = int(prop_days_under_lower_bound * n)
    
#     upper_bound = sorted_rsi[upper_index]
#     lower_bound = sorted_rsi[lower_index]
    
#     return upper_bound, lower_bound

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i]) for i in range(nins)])

    # Calculate dynamic RSI bounds for each stock
    rsi_upper_bounds, rsi_lower_bounds = zip(*[calculate_dynamic_rsi_bounds(rsi, prop_days_over_upper_bound, prop_days_under_lower_bound) 
                                               for rsi in rsi_values])

    # Convert to numpy arrays for easier indexing
    rsi_upper_bounds = np.array(rsi_upper_bounds)
    rsi_lower_bounds = np.array(rsi_lower_bounds)

    # Create the new position array
    new_pos = np.zeros(nins)

    c26_rsi = rsi_values[25]

    RSI_upper_bound = rsi_upper_bounds[25]
    # RSI_lower_bound = rsi_lower_bounds[i]

    # #buy if RSI < RSI_lower_bound
    # if c26_rsi <= RSI_lower_bound:
    #     new_pos[25] = max_pos

    #maintain if RSI < middle bound
    if c26_rsi < RSI_middle_bound:      #changed from elif with above
        # print("maintain long activated")
        new_pos[25] = (RSI_middle_bound-c26_rsi) * multiplier * max_pos / 100

    #short if RSI above upper limit / without shorting this just closes our position
    elif c26_rsi >= RSI_upper_bound:
        new_pos[25] = 0 #-max_pos - no shorting
    
    #maintain if RSI > middle bound
    elif c26_rsi > RSI_middle_bound:
        # print("maintain short activated")
        new_pos[25] = currentPos[25] #(RSI_upper_bound-c26_rsi) * multiplier * max_pos / 100
    
    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = max_pos * change

    # if position_changes[25] != 0:
    #     c26price = prcSoFar[25][-1]
    #     print(f"RSI: {c26_rsi}, price: {c26price}, existing pos: {currentPos[25]}, new_pos: {new_pos[25]}, max_change: +-{max_change}")

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

import pandas as pd

nInst = 50
nt = 500
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.shape
    return df.values.T

pricesFile="./prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(250,501): # 250 in training period here 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] 
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        #print(t)
        #print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)

Loaded 50 instruments for 500 days


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Trading Pos 26 with generic values

In [57]:
# mean(PL): 2.7
# return: 0.00524
# StdDev(PL): 62.18
# annSharpe(PL): 0.68 
# totDvolume: 128011 
# Score: -3.55

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i])[-1] for i in range(nins)])
    c26_rsi = rsi_values[25]

    # Create the new position array
    new_pos = np.zeros(nins)

    #buy if RSI < RSI_lower_bound
    if c26_rsi <= RSI_lower_bound:
        new_pos[25] = 10000

    #maintain if RSI < middle bound
    elif currentPos[25] > 0 and c26_rsi < RSI_middle_bound:
        new_pos[25] = currentPos[25]

    #short if RSI above upper limit
    elif c26_rsi >= RSI_upper_bound:
        new_pos[25] = -5000 #no shorting
    
    #maintain if RSI > middle bound
    elif currentPos[25] < 0 and c26_rsi > RSI_middle_bound:
        new_pos[25] = currentPos[25]
    
    #new_pos[26] = desired_position_26
    
    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = 10000 * change

    if position_changes[25] != 0:
        
        c26price = prcSoFar[25][-1]
        #print(f"RSI: {c26_rsi}, price: {c26price}, existing pos: {currentPos[25]}, new_pos: {new_pos[25]}, max_change: +-{max_change}")

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

### Trading pos 36

In [26]:
# mean(PL): 3.5
# return: 0.00704
# StdDev(PL): 53.94
# annSharpe(PL): 1.04 
# totDvolume: 126360 
# Score: -1.85

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5
multiplier = 3
max_pos = 10000

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i])[-1] for i in range(nins)])
    
    # Create the new position array
    new_pos = np.zeros(nins)

    c36_rsi = rsi_values[35]

    if c36_rsi < RSI_middle_bound:      #changed from elif with above
        # print("maintain long activated")
        new_pos[35] = (RSI_middle_bound-c36_rsi) * multiplier * max_pos / 100

    #short if RSI above upper limit
    elif c36_rsi >= RSI_upper_bound:
        new_pos[35] = 0 #-max_pos - no shorting
    
    #maintain if RSI > middle bound
    elif c36_rsi > RSI_middle_bound:
        # print("maintain short activated")
        new_pos[35] = currentPos[35]

    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = 10000 * change

    # if position_changes[35] != 0:
        
    #     c36price = prcSoFar[35][-1]
    #     #print(f"RSI: {c36_rsi}, price: {c36price}, existing pos: {currentPos[35]}, new_pos: {new_pos[35]}, max_change: +-{max_change}")

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

### Trading pos 42

In [61]:
# mean(PL): 3.2
# return: 0.00775
# StdDev(PL): 36.70
# annSharpe(PL): 1.37 
# totDvolume: 103196 
# Score: -0.48

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i])[-1] for i in range(nins)])
    c36_rsi = rsi_values[41]

    # Create the new position array
    new_pos = np.zeros(nins)

    #buy if RSI < RSI_lower_bound
    if c36_rsi <= RSI_lower_bound:
        new_pos[41] = 10000

    #maintain if RSI < middle bound
    elif currentPos[41] > 0 and c36_rsi < RSI_middle_bound:
        new_pos[41] = currentPos[41]

    #short if RSI above upper limit
    elif c36_rsi >= RSI_upper_bound:
        new_pos[41] = -5000 #no shorting
    
    #maintain if RSI > middle bound
    elif currentPos[41] < 0 and c36_rsi > RSI_middle_bound:
        new_pos[41] = currentPos[41]
    
    #new_pos[26] = desired_position_26
    
    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = 10000 * change

    # if position_changes[41] != 0:
        
    #     c36price = prcSoFar[41][-1]
    #     #print(f"RSI: {c36_rsi}, price: {c36price}, existing pos: {currentPos[41]}, new_pos: {new_pos[41]}, max_change: +-{max_change}")

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

### Trading pos 40

In [63]:
# 

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i])[-1] for i in range(nins)])
    c36_rsi = rsi_values[40]

    # Create the new position array
    new_pos = np.zeros(nins)

    #buy if RSI < RSI_lower_bound
    if c36_rsi <= RSI_lower_bound:
        new_pos[40] = 10000

    #maintain if RSI < middle bound
    elif currentPos[40] > 0 and c36_rsi < RSI_middle_bound:
        new_pos[40] = currentPos[40]

    #short if RSI above upper limit
    elif c36_rsi >= RSI_upper_bound:
        new_pos[40] = -5000 #no shorting
    
    #maintain if RSI > middle bound
    elif currentPos[40] < 0 and c36_rsi > RSI_middle_bound:
        new_pos[40] = currentPos[40]
    
    #new_pos[26] = desired_position_26
    
    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = 10000 * change

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

### Trading pos 26 and 36

In [65]:
# mean(PL): 6.2
# return: 0.00614
# StdDev(PL): 84.05
# annSharpe(PL): 1.17 
# totDvolume: 254371 
# Score: -2.19

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i])[-1] for i in range(nins)])
    
    # Create the new position array
    new_pos = np.zeros(nins)

    c26_rsi = rsi_values[25]

    #buy if RSI < RSI_lower_bound
    if c26_rsi <= RSI_lower_bound:
        new_pos[25] = 10000

    #maintain if RSI < middle bound
    elif currentPos[25] > 0 and c26_rsi < RSI_middle_bound:
        new_pos[25] = currentPos[25]

    #short if RSI above upper limit
    elif c26_rsi >= RSI_upper_bound:
        new_pos[25] = -5000 #no shorting
    
    #maintain if RSI > middle bound
    elif currentPos[25] < 0 and c26_rsi > RSI_middle_bound:
        new_pos[25] = currentPos[25]
    
    c36_rsi = rsi_values[35]

    #buy if RSI < RSI_lower_bound
    if c36_rsi <= RSI_lower_bound:
        new_pos[35] = 10000

    #maintain if RSI < middle bound
    elif currentPos[35] > 0 and c36_rsi < RSI_middle_bound:
        new_pos[35] = currentPos[35]

    #short if RSI above upper limit
    elif c36_rsi >= RSI_upper_bound:
        new_pos[35] = -5000 #no shorting
    
    #maintain if RSI > middle bound
    elif currentPos[35] < 0 and c36_rsi > RSI_middle_bound:
        new_pos[35] = currentPos[35]
    
    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = 10000 * change

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

### Trading all pos

In [24]:
# mean(PL): 14.5
# return: 0.00230
# StdDev(PL): 160.67
# annSharpe(PL): 1.43 
# totDvolume: 1586866 
# Score: -1.55

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.1
threshold = 0.02
multiplier = 4
max_pos = 10000

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i]) for i in range(nins)])
    # prev_return_22 = prcSoFar[22, -1] / prcSoFar[22, -2] - 1
    # prev_return_30 = prcSoFar[30, -1] / prcSoFar[30, -2] - 1
    # prev_return_11 = prcSoFar[11, -1] / prcSoFar[11, -2] - 1
    
    # Create the new position array
    new_pos = np.zeros(nins)

    # # Determine the desired position for company 38
    # if prev_return_22 > threshold and prev_return_30 > threshold: #  and prev_return_25 > threshold
    #     desired_position_38 = 10000
    # elif prev_return_22 < threshold and prev_return_30 < threshold: # and prev_return_25 < threshold
    #     desired_position_38 = -10000
    # else:
    #     desired_position_38 = 0
    
    # # Determine the desired position for company 38
    # if prev_return_22 > threshold and prev_return_11 > threshold: #  and prev_return_38 > threshold
    #     desired_position_27 = 10000
    # elif prev_return_22 < threshold and prev_return_11 < threshold: # and prev_return_38 < threshold
    #     desired_position_27 = -10000
    # else:
    #     desired_position_27 = 0

    for i in range(nins):
        rsi = rsi_values[i]

        # Buy if RSI < RSI_lower_bound
        if rsi < RSI_lower_bound:
            new_pos[i] = max_pos

        # Maintain if RSI < middle bound and current position is positive
        if rsi < RSI_middle_bound:
            new_pos[i] = (RSI_middle_bound-rsi) * multiplier * max_pos / (max_pos / 100)

        # Short if RSI >= RSI_upper_bound
        elif rsi >= RSI_upper_bound:
            new_pos[i] = 0  # No shorting

        # Maintain if RSI > middle bound and current position is negative
        elif rsi > RSI_middle_bound:
            new_pos[i] = currentPos[i]

    # new_pos[38] = desired_position_38
    # new_pos[27] = desired_position_27

    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = max_pos * change

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

In [31]:
# mean(PL): 14.5
# return: 0.00230
# StdDev(PL): 160.67
# annSharpe(PL): 1.43 
# totDvolume: 1586866 
# Score: -1.55

# RSI_upper_bound = 70
# RSI_lower_bound = 35
# RSI_middle_bound = 50
# change = 0.5

import numpy as np

nInst = 50
currentPos = np.zeros(nInst)
RSI_upper_bound = 70
RSI_lower_bound = 35
RSI_middle_bound = 50
change = 0.5
threshold = 0.02
prop_days_over_upper_bound = 0.1
prop_days_under_lower_bound = 0.05

def calculate_rsi(prices, period=14):
    delta = np.diff(prices)
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(prices)
    avg_loss = np.zeros_like(prices)

    avg_gain[period] = np.mean(gain[:period])
    avg_loss[period] = np.mean(loss[:period])

    for i in range(period + 1, len(prices)):
        avg_gain[i] = (avg_gain[i - 1] * (period - 1) + gain[i - 1]) / period
        avg_loss[i] = (avg_loss[i - 1] * (period - 1) + loss[i - 1]) / period

    rs = avg_gain / (avg_loss + 0.00000000001)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_dynamic_rsi_bounds(rsi_values, prop_days_over_upper_bound, prop_days_under_lower_bound):
    sorted_rsi = np.sort(rsi_values)
    n = len(sorted_rsi)
    
    upper_index = int((1 - prop_days_over_upper_bound) * n)
    lower_index = int(prop_days_under_lower_bound * n)
    
    upper_bound = sorted_rsi[upper_index]
    lower_bound = sorted_rsi[lower_index]
    
    return upper_bound, lower_bound

def getPosition(prcSoFar):
    global currentPos
    (nins, nt) = prcSoFar.shape

    if (nt < 2):
        return np.zeros(nins)
    
    #calculate current day RSI for each stock
    rsi_values = np.array([calculate_rsi(prcSoFar[i]) for i in range(nins)])
    # prev_return_22 = prcSoFar[22, -1] / prcSoFar[22, -2] - 1
    # prev_return_30 = prcSoFar[30, -1] / prcSoFar[30, -2] - 1
    # prev_return_11 = prcSoFar[11, -1] / prcSoFar[11, -2] - 1
    
    # Calculate dynamic RSI bounds for each stock
    rsi_bounds = [calculate_dynamic_rsi_bounds(rsi, prop_days_over_upper_bound, prop_days_under_lower_bound) 
                  for rsi in rsi_values]

    # Create the new position array
    new_pos = np.zeros(nins)

    # # Determine the desired position for company 38
    # if prev_return_22 > threshold and prev_return_30 > threshold: #  and prev_return_25 > threshold
    #     desired_position_38 = 10000
    # elif prev_return_22 < threshold and prev_return_30 < threshold: # and prev_return_25 < threshold
    #     desired_position_38 = -10000
    # else:
    #     desired_position_38 = 0
    
    # # Determine the desired position for company 38
    # if prev_return_22 > threshold and prev_return_11 > threshold: #  and prev_return_38 > threshold
    #     desired_position_27 = 10000
    # elif prev_return_22 < threshold and prev_return_11 < threshold: # and prev_return_38 < threshold
    #     desired_position_27 = -10000
    # else:
    #     desired_position_27 = 0

    for i in range(nins):
        rsi = rsi_values[i][-1]
        RSI_upper_bound, RSI_lower_bound = rsi_bounds[i]

        # Buy if RSI < RSI_lower_bound
        if rsi <= RSI_lower_bound:
            new_pos[i] = 0

        # Maintain if RSI < middle bound and current position is positive
        elif rsi < RSI_middle_bound:
            new_pos[i] = (RSI_middle_bound-rsi) * multiplier * max_pos / 100

        # Short if RSI >= RSI_upper_bound
        elif rsi >= RSI_upper_bound:
            new_pos[i] = 0  # No shorting

        # Maintain if RSI > middle bound and current position is negative
        elif rsi > RSI_middle_bound:
            new_pos[i] = currentPos[i]

    # new_pos[38] = desired_position_38
    # new_pos[27] = desired_position_27

    # Calculate the change in positions
    position_changes = new_pos - currentPos

    # Apply a maximum change of 20% of the maximum allowed position
    max_change = 10000 * change

    position_changes = np.clip(position_changes, -max_change, max_change)
    
    # Update current positions
    currentPos += position_changes.astype(int)

    return currentPos

### Evaluation

In [32]:
#!/usr/bin/env python

import numpy as np
import pandas as pd
# from teamName import getMyPosition

nInst = 50
nt = 500
commRate = 0.0010
dlrPosLimit = 10000

def loadPrices(fn):
    global nt, nInst
    df=pd.read_csv(fn, sep='\s+', header=None, index_col=None)
    nt, nInst = df.shape
    return df.values.T

pricesFile="./prices.txt"
prcAll = loadPrices(pricesFile)
print ("Loaded %d instruments for %d days" % (nInst, nt))

def calcPL(prcHist):
    cash = 0
    curPos = np.zeros(nInst)
    totDVolume = 0
    totDVolumeSignal = 0
    totDVolumeRandom = 0
    value = 0
    todayPLL = []
    (_,nt) = prcHist.shape
    for t in range(250,501): # 250 in training period here 
        prcHistSoFar = prcHist[:,:t]
        newPosOrig = getPosition(prcHistSoFar)
        curPrices = prcHistSoFar[:,-1] 
        posLimits = np.array([int(x) for x in dlrPosLimit / curPrices])
        newPos = np.clip(newPosOrig, -posLimits, posLimits)
        deltaPos = newPos - curPos
        dvolumes = curPrices * np.abs(deltaPos)
        dvolume = np.sum(dvolumes)
        totDVolume += dvolume
        comm = dvolume * commRate
        cash -= curPrices.dot(deltaPos) + comm
        curPos = np.array(newPos)
        posValue = curPos.dot(curPrices)
        todayPL = cash + posValue - value
        todayPLL.append(todayPL)
        value = cash + posValue
        ret = 0.0
        if (totDVolume > 0):
            ret = value / totDVolume
        #print ("Day %d value: %.2lf todayPL: $%.2lf $-traded: %.0lf return: %.5lf" % (t,value, todayPL, totDVolume, ret))
    pll = np.array(todayPLL)
    (plmu,plstd) = (np.mean(pll), np.std(pll))
    annSharpe = 0.0
    if (plstd > 0):
        annSharpe = np.sqrt(250) * plmu / plstd
    return (plmu, ret, plstd, annSharpe, totDVolume)

(meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
score = meanpl - 0.1*plstd
print ("=====")
print ("mean(PL): %.1lf" % meanpl)
print ("return: %.5lf" % ret)
print ("StdDev(PL): %.2lf" % plstd)
print ("annSharpe(PL): %.2lf " % sharpe)
print ("totDvolume: %.0lf " % dvol)
print ("Score: %.2lf" % score)

Loaded 50 instruments for 500 days
=====
mean(PL): -45.3
return: -0.00151
StdDev(PL): 390.79
annSharpe(PL): -1.83 
totDvolume: 7528209 
Score: -84.35


### GridSearch Eval

In [137]:
def run_evaluation(threshold, change):
    global currentPos
    currentPos = np.zeros(nInst)  # Reset currentPos for each evaluation
    max_pos_change = 10000 * change
    
    (meanpl, ret, plstd, sharpe, dvol) = calcPL(prcAll)
    score = meanpl - 0.1*plstd
    return {
        'threshold': threshold,
        'change': change,
        'mean_pl': meanpl,
        'return': ret,
        'std_dev': plstd,
        'ann_sharpe': sharpe,
        'tot_dvolume': dvol,
        'score': score
    }

In [138]:
import itertools
import pandas as pd

# Define the ranges for threshold and change
thresholds = np.arange(0.002, 0.031, 0.002)  # 0.001 to 0.02 in steps of 0.001
changes = np.arange(0.02, 0.51, 0.04)  # 0.05 to 0.5 in steps of 0.05

# Perform grid search
results = []
for threshold, change in itertools.product(thresholds, changes):
    result = run_evaluation(threshold, change)
    results.append(result)

# Convert results to DataFrame
df_results = pd.DataFrame(results)

# Sort results by score in descending order
df_results_sorted = df_results.sort_values('score', ascending=False)

# Display top 10 results
print("Top 10 Results:")
print(df_results_sorted.head(10))

# Save all results to CSV
df_results.to_csv('grid_search_results.csv', index=False)
print("\nAll results saved to 'grid_search_results.csv'")

# # Plot heatmap of scores
# import matplotlib.pyplot as plt
# import seaborn as sns

# pivot_table = df_results.pivot('threshold', 'change', 'score')
# plt.figure(figsize=(12, 8))
# sns.heatmap(pivot_table, annot=False, cmap='YlOrRd', fmt='.2f')
# plt.title('Score Heatmap')
# plt.xlabel('Change')
# plt.ylabel('Threshold')
# plt.show()

Top 10 Results:
     threshold  change   mean_pl    return   std_dev  ann_sharpe  tot_dvolume  \
97       0.016    0.26  2.801271  0.030583  13.91089     3.18398     22990.89   
146      0.024    0.14  2.801271  0.030583  13.91089     3.18398     22990.89   
124      0.020    0.30  2.801271  0.030583  13.91089     3.18398     22990.89   
125      0.020    0.34  2.801271  0.030583  13.91089     3.18398     22990.89   
126      0.020    0.38  2.801271  0.030583  13.91089     3.18398     22990.89   
127      0.020    0.42  2.801271  0.030583  13.91089     3.18398     22990.89   
128      0.020    0.46  2.801271  0.030583  13.91089     3.18398     22990.89   
129      0.020    0.50  2.801271  0.030583  13.91089     3.18398     22990.89   
130      0.022    0.02  2.801271  0.030583  13.91089     3.18398     22990.89   
131      0.022    0.06  2.801271  0.030583  13.91089     3.18398     22990.89   

        score  
97   1.410182  
146  1.410182  
124  1.410182  
125  1.410182  
126  1.41018